In [2]:
import cdt
import networkx as nx
import warnings
import numpy as np
import os
from cdt.causality.graph import CAM
from pandas import DataFrame

warnings.filterwarnings("ignore")  # silence warnings

cdt.SETTINGS.rpath = 'C:\Program Files\R\R-4.2.1\\bin\Rscript' # path to your r executable
cdt.SETTINGS.rpath

Detecting 1 CUDA device(s).


'C:\\Program Files\\R\\R-4.2.1\\bin\\Rscript'

In [3]:
OUTPUT_DIR = "outputs"
DATA_DIR = "data"
PATH_SPLITTER = "\\"

In [4]:
if not os.path.isdir(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)
notear_path = os.path.join(OUTPUT_DIR, 'cam')
if not os.path.isdir(notear_path):
    os.mkdir(notear_path)

In [5]:
data_mechanism_dirs = os.listdir(path=DATA_DIR)
data_mechanism_dirs

['gaussian_add_mechanism',
 'gaussian_mix_mechanism',
 'linear_mechanism',
 'mix_mechanism']

# Process data

In [6]:
max_depth = 0
for root, directories, files in os.walk(DATA_DIR):
    if len(root.split(PATH_SPLITTER)) > max_depth:
        max_depth = len(root.split(PATH_SPLITTER))

In [7]:
data_dirs = [x[0].replace(DATA_DIR, "") for x in os.walk(DATA_DIR) if len(x[0].split(PATH_SPLITTER)) > max_depth-1 ]
output_dirs = [x[0].replace(notear_path, "") for x in os.walk(notear_path) if len(x[0].split(PATH_SPLITTER)) > max_depth]

In [8]:
for data_dir in data_dirs:
    data_path = data_dir.split(PATH_SPLITTER)[1:]
    mechanism = data_path[0]
    dataset = data_path[1]

    input_mechanism_path = os.path.join(DATA_DIR, mechanism)
    output_mechanism_path = os.path.join(notear_path, mechanism)
    # Create results dir each causal mechanism group of data
    if not os.path.isdir(output_mechanism_path):
        os.mkdir(output_mechanism_path)

    # Create results dir each dataset
    input_dataset_path = os.path.join(input_mechanism_path, dataset)
    output_dataset_path = os.path.join(output_mechanism_path, dataset)
    if not os.path.isdir(output_dataset_path):
        os.mkdir(output_dataset_path)
    
    if len(os.listdir(output_dataset_path)) == 0:
        dataset_path = os.path.join(input_dataset_path, 'data1.npy')

        # Process
        data = np.load(dataset_path)
        obj = CAM()
        pred_graph = obj.predict(DataFrame(data))

        pred_path = os.path.join(output_dataset_path, "pred.npy")
        np.save(pred_path, nx.to_numpy_array(pred_graph))  
